# 공공데이타포탈의 API 서비스 이용하기 

- https://www.data.go.kr/
- 회원 가입후에 API서비스 신청 

### 공공데이터활용지원센터_보건복지부 코로나19 감염 현황

- https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15043376
- [활용신청] 클릭후 서비스키 정보 확인
[마이페이지]-[오픈 API]-[개발 계정]

```
일반 인증키 (Encoding)
vPgcMKBzk6nt0FAdWvw3xagcCcykHB2FQixNBvAifCU11%2BIPy8YJtecAA1tDqPZgV%2FJA51kOgHp%2B4XGss7ECUQ%3D%3D

일반 인증키(Decoding)	
vPgcMKBzk6nt0FAdWvw3xagcCcykHB2FQixNBvAifCU11+IPy8YJtecAA1tDqPZgV/JA51kOgHp+4XGss7ECUQ==

```

In [2]:
# (1) 관련 모듈 임포트 

from xml.etree.ElementTree import *
import pandas as pd
import requests

In [16]:
# 2) 페이지 요청 

url = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson'
# 서비스키 설정 - 디코딩한 서비스키 이용 
serviceKey = 'Yhq6n9vlpaKZsCOw0jec8zIdZ8p+Bpuku6WLjVgDhqXRW6dHnfXoauSEj19jpnjv59CUGspyTxVCgCeXOua7dg=='

params ={'serviceKey' : serviceKey, 
         'pageNo' : '1', 
         'numOfRows' : '10', 
         'startCreateDt' : '20210101', # 시작일
         'endCreateDt' : '20211021' } # 종료일

response = requests.get(url, params=params)

print(type(response.text))
# print(response.text)
print(url)

<class 'str'>
http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson


In [22]:
# (3) xml 데이타 소스 => 파이썬의 xml 객체 
# xml객체명 = ElementTree(fromstring(xml데이타소스))

tree = ElementTree(fromstring(response.text))
tree

In [37]:
# (4) 최상위 부모 태그 요소 찾기 와 탐색 
# root = xml객체.getroot()
root = tree.getroot()
root, type(root)

(<Element 'response' at 0x000001CCFF9070E0>, xml.etree.ElementTree.Element)

In [31]:
body = root.find('body')
items = root.find('body').find('items')
item_list = root.find('body').find('items').findall('item')
print(body)
print(items)
print(len(item_list))

<Element 'body' at 0x000001CCFF907270>
<Element 'items' at 0x000001CCFF9072C0>
294


In [36]:
item_list[0].find('stateDt').tag,  item_list[0].find('stateDt').text

('stateDt', '20211021')

In [39]:
# (5) 2차원 리스트 저장 
# 기준일 STATE_DT
# 확진자 수 DECIDE_CNT
# 검사진행수  EXAM_CNT
# 사망자 수 DEATH_CNT

result_list = []
for item in item_list:
    stateDt = item.find('stateDt').text
    decideCnt= item.find('decideCnt').text
    examCnt= item.find('examCnt').text
    deathCnt= item.find('deathCnt').text
    result_list.append([stateDt, decideCnt, examCnt, deathCnt])
    

In [42]:
len(result_list), result_list[0], result_list[-1]

(294,
 ['20211021', '347529', '1334586', '2709'],
 ['20210101', '61753', '176927', '917'])

In [43]:
# (6) 데이타프레임 변경
df_covid = pd.DataFrame(result_list, columns=['기준일', '확진자수', '검사진행수','사망자수'])
df_covid

,기준일,확진자수,검사진행수,사망자수
0,20211021,347529,1334586,2709
1,20211020,346088,1325132,2698
2,20211019,344517,1323771,2689
3,20211018,343445,1320566,2668
4,20211017,342396,1312433,2660
...,...,...,...,...
289,20210105,64964,193751,1007
290,20210104,64250,190732,981
291,20210103,63230,186472,962
292,20210102,62573,179387,942
